## Chapter 03. 평가

In [1]:
# 모듈 로딩
import pandas as pd
import numpy as np

- 분류의 성능 평가 지표
    * 정확도 (Accuracy)             
        - 실제 데이터에서 예측 데이터가 얼마나 같은지 판단하는 지표 (예측 결과가 동일한 데이터 건수 / 전체 예측 데이터 건수)  
        - 불균형한 레이블 값 분포에서 성능을 판단할 때 적합한 평가지표가 아님 
    * 오차행렬(Confusion Matrix)
        - 이진 분류의 예측 오류가 얼마인지와 더불어 어떠한 유형의 예측 오류가 발생하고 있는지를 함께 나타내는 지표
    * 정밀도 (Precision)
        - 예측을 Positive로 한 대상 중에 예측과 실제 값이 Positive로 일치한 데이터의 비율
        - 실제 Negative 음성 데이터를 Positive로 잘못 판단하게 되면 큰 영향이 발생할 때 중요한 지표가 됨
    * 재현율 (Recall) (= 민감도, TP Rate)
        - 실제 값이 Positive인 대상 중에 예측과 실제 값이 Positive로 일치한 데이터의 비율
        - 실제 Positive 양성 데이터를 Negative로 잘못 판단하게 되면 큰 영향이 발생할 때 중요한 지표가 됨
    * F1 스코어
    * ROC AUC

#### 정밀도와 재현율

- 2단원에서 정의한 함수 불러오기

In [2]:
# 문자열 카테고리 피쳐를 숫자형 카테고리 피처로 변환 - 사이킷런 LabelEncoder 이용
from sklearn.preprocessing import LabelEncoder

# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(), inplace = True)
    df['Cabin'].fillna('N', inplace = True)
    df['Embarked'].fillna('N', inplace = True)
    df['Fare'].fillna(0, inplace = True)
    return df

# 머신러닝 알고리즘에 불필요한 피처 제거
def drop_features(df):
    df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace = True)
    return df

# 레이블 인코딩 수행
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])

    return df

# 앞에서 설정한 데이터 전처리 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [3]:
# 함수 정의 - confusion matrix, accuracy, precision, recall 평가 한번에 호출
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test,pred)
    precision = precision_score(y_test,pred)
    recall = recall_score(y_test,pred)

    print('오차행렬')
    print(confusion)
    print(f'정확도 : {accuracy:.4f}, 정밀도 : {precision:.4f}, 재현율 : {recall:.4f}')

In [4]:
# 로지스틱 회귀 기반으로 타이타닉 생존자 예측, confusion matrix, accuracy, precision, recall 평가 수행
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# 원본데이터를 로딩
titanic_df = pd.read_csv('../DAY46/titanic_train.csv')

# 데이터 가공
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis=1)
X_titanic_df = transform_features(X_titanic_df)

# 학습데이터/테스트데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, test_size=0.2,random_state=11)

# 로지스틱회귀 인스턴스 생성
lr_clf = LogisticRegression(solver='liblinear')
# 학습 진행
lr_clf.fit(X_train, y_train)
# 예측
pred = lr_clf.predict(X_test)
# 평가 출력
get_clf_eval(y_test,pred)

오차행렬
[[108  10]
 [ 14  47]]
정확도 : 0.8659, 정밀도 : 0.8246, 재현율 : 0.7705


#### 정밀도/재현율 트레이드오프
- 정밀도와 재현율은 상호 보완적인 지표 => 한쪽의 값이 높아지면 다른 쪽의 수치는 떨어짐 == 정밀도/재현율의 트레이드오프(Trade-off)

In [8]:
# predict_proba()와 predict() 반환값 비교
pred_proba = lr_clf.predict_proba(X_test)
pred = lr_clf.predict(X_test)

print(f'pred_proba()결과 Shape : {pred_proba.shape}')
print(f'pred_proba array에서 앞 3개만 샘플로 추출 \n: {pred_proba[:3]}')

# 예측 확률 array와 예측 결괏값 array를 병합(concatenate)해 예측 확률과 결괏값을 한눈에 확인
pred_proba_result = np.concatenate([pred_proba, pred.reshape(-1,1)],axis=1)
print(f'두 개의 class 중에서 더 큰 확률을 클래스 값으로 예측 \n{pred_proba_result[:3]}')

pred_proba()결과 Shape : (179, 2)
pred_proba array에서 앞 3개만 샘플로 추출 
: [[0.44935225 0.55064775]
 [0.86335511 0.13664489]
 [0.86429643 0.13570357]]
두 개의 class 중에서 더 큰 확률을 클래스 값으로 예측 
[[0.44935225 0.55064775 1.        ]
 [0.86335511 0.13664489 0.        ]
 [0.86429643 0.13570357 0.        ]]


In [9]:
# 임계치를 수정해 예측값 조절
from sklearn.preprocessing import Binarizer
X = [[1,-1,2],
     [2,0,0],
     [0,1.1,1.2]]

# Binarizer객체의 fit_transform : X의 개별 원소들이 threshold값보다 같거나 작으면 0을, 크면 1을 반환
binarizer = Binarizer(threshold=1.1)
print(binarizer.fit_transform(X))

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]]


In [10]:
from sklearn.preprocessing import Binarizer

# Binarizer의 임계치 설정. 분류 결정 임곗값
custom_threshold = 0.5

# predict_proba() 반환값의 두번째 칼럼, Positive 클래스 칼럼만 추출해 Binarizer 적용
pred_proba1 = pred_proba[:,1].reshape(-1,1)

binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba1)
custom_predict = binarizer.transform(pred_proba1)

get_clf_eval(y_test, custom_predict)
# 로지스틱 회귀분류 객체에서 호출된 predict() 지표값과 같음 => predict()는 predict_proba()에 기반함

오차행렬
[[108  10]
 [ 14  47]]
정확도 : 0.8659, 정밀도 : 0.8246, 재현율 : 0.7705


In [11]:
## 임곗값 낮추기
# Binarizer의 분류결정 임곗값을 0.4로 변경
custom_threshold = 0.4

pred_proba1 = pred_proba[:,1].reshape(-1,1)

binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba1)
custom_predict = binarizer.transform(pred_proba1)

get_clf_eval(y_test, custom_predict)
# 재현율이 올라가고 정밀도가 떨어짐

오차행렬
[[97 21]
 [11 50]]
정확도 : 0.8212, 정밀도 : 0.7042, 재현율 : 0.8197


- Positive 예측값이 많아지면 상대적으로 재현울 값이 높아짐 ==> 정밀도는 하락

In [13]:
## 임곗값을 0.4에서 0.6까지 0.05씩 증가시키며 평가지표 조사 - get_eval_by_threshold() 함수 생성
# 테스트를 수행할 모든 임곗값을 리스트 객체로 저장
thresholds = [0.4,0.45,0.50,0.55,0.60]

def get_eval_by_threshold(y_test, pred_proba_c1, thresholds):
    # thresholds list 객체 내의 값을 차례로 iteration하면서 Evaluation 수행
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print('임곗값 :', custom_threshold)
        get_clf_eval(y_test, custom_predict)

get_eval_by_threshold(y_test, pred_proba[:,1].reshape(-1,1), thresholds)

임곗값 : 0.4
오차행렬
[[97 21]
 [11 50]]
정확도 : 0.8212, 정밀도 : 0.7042, 재현율 : 0.8197
임곗값 : 0.45
오차행렬
[[105  13]
 [ 13  48]]
정확도 : 0.8547, 정밀도 : 0.7869, 재현율 : 0.7869
임곗값 : 0.5
오차행렬
[[108  10]
 [ 14  47]]
정확도 : 0.8659, 정밀도 : 0.8246, 재현율 : 0.7705
임곗값 : 0.55
오차행렬
[[111   7]
 [ 16  45]]
정확도 : 0.8715, 정밀도 : 0.8654, 재현율 : 0.7377
임곗값 : 0.6
오차행렬
[[113   5]
 [ 17  44]]
정확도 : 0.8771, 정밀도 : 0.8980, 재현율 : 0.7213


In [ ]:
from sklearn.metrics import precision_recall_curve

# 레이블 값이 1일때의 예측확률을 추출
pred_proba_class1 = lr_clf.predict_proba(X_test)[:,1]

# 실제값 데이터세트와 레이블 값이 1일때의 예측확률을 precision_recall_curve 인자로 입력
precisions, recalls, thresholds = precision_recall_curve(y_test, pred_proba_class1)
print('반환된 분류 결정 임곗값 배열의 Shape:', thresholds.shape)

# precision_recall_curve 이해가 안됨.....ㅠ